# LangChain Use Cases

This document dives straight into implementing [LangChain use case](https://docs.langchain.com/docs/category/use-cases) using a step-by-step approach, and hopefully inspire you to build.

## Summarization

Langchain summarization is one of the most common usage of LangChain and LLMs. It can summarize any amount of text or documentations, including the ones that exceeds the token limit (currently set at `4096 tokens` for `gpt-35-turbo`), which roughly equates to `3000 words`. 

The technique used is similar to the concept of `sliding window`, which a fixed size window, usually under the max context window limit, is used to chunk the the long document into smaller pieces, then summarize the content recursively to produce the final summary using mapreduce.

You can use it to summarize not only text, books, documents, audios, social media threads, and etc. Some use cases including: _articles_, _research pagers_, _legal and financial documents_, _transcripts_, _chat history_, _customer interactions_, _product reviews_, _podcasts_, _twitter threads_ and more.

In [4]:
import dotenv
import os
import pdfplumber
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

Now, let's load up `The Tragedy of Hamlet`

In [5]:
with pdfplumber.open("hamlet.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
        
print(text[:300])

The Tragedy of Hamlet, Prince of
Denmark
ASCIItextplacedinthepublicdomainbyMobyLexicalTools,1992.SGMLmarkupbyJonBosak,
1992-1994.XMLversionbyJonBosak,1996-1999.SimplifiedXMLversionbyMaxFroumentin,2001.The
XMLmarkupinthisversionisCopyright©1999JonBosak.Thisworkmayfreelybedistributedoncondition
thatit


By outputting the total number of tokens of the book with [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens), you can clearly see it exceeds the context window limit.

In [7]:
n_tokens = llm.get_num_tokens(text)

print(f"total number of tokens: {n_tokens}")

total number of tokens: 53929


The process starts by `splitting` the text into multiple parts by `chunk size` using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html), with a `coverlap` of each adjacent chunk of text.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2000, chunk_overlap=200, length_function = len)
docs = text_splitter.create_documents([text])

print(f"number of docs: {len(docs)}")

number of docs: 100


Now the docs are ready, we can then load up a `chain` to do produce the sum of sums.

In [12]:
chain = load_summarize_chain(llm = llm, chain_type = 'map_reduce')
result = chain.run(docs)

print(result)

 In The Tragedy of Hamlet, Prince of Denmark, Hamlet is on a quest for revenge against his uncle, Claudius, who has usurped the throne. After encountering a ghostly figure, Hamlet learns that his father was murdered by his uncle and is determined to avenge his death. King Claudius and Lord Polonius devise a plan to spy on Hamlet and Ophelia, and Hamlet kills Polonius. Laertes returns from France and challenges Hamlet to a fencing match, during which Laertes wounds Hamlet with a poisoned sword. Hamlet stabs King Claudius with a poisoned sword and forces him to drink a poisoned potion. Prince Fortinbras arrives and orders that the bodies be placed on a stage for all to see, and then claims his rights of memory in the kingdom.


When instanciating the `chain`, you can also add `Verbose = True` to see the steps LangChain took before producing the final summary, i.e. generating a summary per each chunk of text, and produce the final summary by aggregating the 100 summaries with map reduce.

This is a powerful technique to overcome the token limitation. Even though more powerful models will be released in the future to support more tokens, such as Claude now supports [100K tokens](https://www.anthropic.com/index/100k-context-windows) as of 11 May 2023, biggest of the its kind. There is not guaranteed better performance, accuracy or cost-effectiveness.

Moreover, you can [parallelise the calls](https://github.com/hwchase17/langchain/issues/1073) to super charge summarization using `batch_size` when instanciating LLMs.

## Question Answering Over Documents

Similiar to how humans answer questions, first you need to provide some context to LLMs. The context can be in many different formats, such as text documents, SQL database, APIs, etc. For the purpose of this exercise, we will focus on dealing with text documents input as context. 

However, as the context grows, it naturally exceeds the token limit (again, just like in the case of summarization), and it gets harder and harder to give accurate answer quickly. The problem is solved by converting the context and your question into [embeddings](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings), basically a list of vector value representaiton of information; then find out a few results that are closely related to your question before giving out a final answer. The process to do the comparison is via an algorithm called [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). If you are getting confused by the description, don't worry. The formular is as following: 

`llm(context + question) = your answer`.

With this capability, you can _chat with your documents_, _ask questions to papers_, _create study guides_, _reference medical information_ and more.

In [15]:
from langchain import OpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

Now, let's load `The Tragedy of Hamlet` again.

In [18]:
loader = PyMuPDFLoader("hamlet.pdf")
doc = loader.load()
print(f"number of docs loaded: {len(docs)}")
print(f"number of characters in the first doc: {len(doc[0].page_content)}")

number of docs loaded: 142
number of characters in the first doc: 389


Now, we can split the PDF into chunks based on our definition.

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
docs = text_splitter.split_documents(doc)

Then convert the documents into embeddings by calling OpenAI API, and store the result in a local vector store called `FAISS`. You can choose from a number of supported [vector stores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) (local or remote).

In [22]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
knowledge_base = FAISS.from_documents(docs, embeddings)

Create a retrieval QA engine to perform queries.

In [27]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "refine", retriever = knowledge_base.as_retriever())
query = """What is the subject of Hamlet's second soliloquy, the famous "To be or not to be" speech?"""
qa.run(query)

'\n\nThe subject of Hamlet\'s second soliloquy, the famous "To be or not to be" speech, is the contemplation of life and death and the decision of whether to take action against the troubles of life or to accept them. Hamlet is reflecting on the consequences of his inaction and the potential consequences of taking action, as well as the idea of death as a possible escape from his troubles. He is also considering the idea of suicide as a way to end his suffering, and is presented with a supernatural being that beckons him to follow it. Hamlet questions the fear of death and the power of his immortal soul, and ultimately decides to follow the ghost. He also reflects on the idea of ambition and how it can be a prison, and how even a monarch can be confined in a "nut shell" if their ambition is too great.'

By now, you should have a working example of QA over documents. You should also notice there are a number of similarities comparing to `summarization`, such as the way the context was loaded, splitted, before passing onto the `chain`.

You may also have noticed different `chain types` specification used, and there is a good reason for that. Before explaining further, understand that LangChain chain types are basically different ways you can connect LLMs. Here are the supported chain types:
- `map_reduce`: It separates texts into batches, feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.
- `map-rerank`: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.
- `refine`: It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.
- `stuff`: The default chain type that uses ALL of the text from the documents in the prompt.